In [2]:
import requests
import pandas as pd

class StockDataFetcher:
    def __init__(self):
        self.api_key = "PKW0YNTXIQBYEO0GHQS4"
        self.api_secret = "r0obc1M6nmK1k9aTF7OixLHtGB0PCLBR9RFAJ2Rm"
        self.base_url = "https://data.alpaca.markets/v2/stocks/bars"
    
    def get_stock_data(self, symbol: str, start_date: str, end_date: str,timeframe: str = "1Min", limit: int = 5000):
        url = f"{self.base_url}?symbols={symbol}&timeframe={timeframe}&start={start_date}&end={end_date}&limit={limit}&adjustment=raw&feed=sip&sort=asc"
        
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": self.api_key,
            "APCA-API-SECRET-KEY": self.api_secret
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            return self.create_dataframe(data, symbol)
        else:
            return {"error": f"Request failed with status code {response.status_code}"}
    
    def create_dataframe(self, data, symbol):
        if "bars" not in data or symbol not in data["bars"]:
            return pd.DataFrame()
        
        df = pd.DataFrame(data["bars"][symbol])
        df["t"] = pd.to_datetime(df["t"])
        df.rename(columns={"t": "time", "o": "open", "h": "high", "l": "low", "c": "close", "v": "volume", "vw": "vwap", "n": "trades"}, inplace=True)
        return df



In [3]:

fetcher = StockDataFetcher()

stock_symbol = "TSLA"
start_date = "2025-02-25T00:00:00Z"
end_date = "2025-03-02T09:30:00-04:00"

df = fetcher.get_stock_data(stock_symbol, start_date, end_date)



In [4]:
df

,close,high,low,trades,open,time,volume,vwap
0,327.0500,327.0500,326.9000,94,326.9811,2025-02-25 00:00:00+00:00,2175,326.998533
1,327.2000,327.2200,327.0500,71,327.0600,2025-02-25 00:01:00+00:00,1644,327.132968
2,327.1600,327.2500,327.1000,83,327.1500,2025-02-25 00:02:00+00:00,3368,327.211843
3,327.2500,327.2500,327.0800,65,327.1700,2025-02-25 00:03:00+00:00,1367,327.161614
4,326.8218,327.2500,326.8000,176,327.2500,2025-02-25 00:04:00+00:00,6807,326.999470
...,...,...,...,...,...,...,...,...
3874,293.8000,293.8000,293.5000,94,293.6900,2025-03-01 00:55:00+00:00,2423,293.694392
3875,294.1365,294.1800,293.8001,287,293.8100,2025-03-01 00:56:00+00:00,12443,293.922855
3876,294.1187,294.1365,293.9152,154,294.1365,2025-03-01 00:57:00+00:00,5124,294.068604
3877,294.1507,294.1800,294.0500,149,294.1200,2025-03-01 00:58:00+00:00,5991,294.121301


In [5]:
import pandas as pd
import ta

# Load your stock data
data = df

# Calculate technical indicators
data['MA50'] = ta.trend.sma_indicator(data['close'], window=30)
data['RSI'] = ta.momentum.rsi(data['close'], window=30)
data['MACD'] = ta.trend.macd(data['close'])
data['BB_upper'] = ta.volatility.bollinger_hband(data['close'])
data['BB_lower'] = ta.volatility.bollinger_lband(data['close'])
data['ADX'] = ta.trend.adx(data['high'], data['low'], data['close'])
data['CCI'] = ta.trend.cci(data['high'], data['low'], data['close'])
data['ATR'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'])
data['ROC'] = ta.momentum.roc(data['close'])
data['OBV'] = ta.volume.on_balance_volume(data['close'], data['volume'])

# Drop rows with NaN values (resulting from the indicator calculation)
data = data.dropna()

# Add more indicators as needed

# Drop rows with NaN values (resulting from the indicator calculation)
data

,close,high,low,trades,open,time,volume,vwap,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
29,326.6500,326.8200,326.6000,199,326.6000,2025-02-25 00:29:00+00:00,6945,326.714519,326.158410,53.434353,-0.108864,326.627259,325.041671,37.074390,188.512127,0.351196,0.144307,-12199
30,326.9122,327.0000,326.5200,133,326.5300,2025-02-25 00:30:00+00:00,8833,326.892102,326.153817,56.748169,-0.025835,326.793594,324.958556,36.909908,188.643173,0.360396,0.372183,-3366
31,326.7800,326.9600,326.7800,111,326.9600,2025-02-25 00:31:00+00:00,3785,326.864655,326.139817,54.717176,0.028964,326.921779,324.926371,36.757174,159.341735,0.347511,0.454965,-7151
32,326.8500,327.0000,326.6100,99,326.6100,2025-02-25 00:32:00+00:00,2919,326.866749,326.129483,55.587835,0.077152,327.049408,324.899742,35.914564,131.886869,0.350546,0.457954,-4232
33,326.5000,326.9000,326.5000,140,326.9000,2025-02-25 00:33:00+00:00,3871,326.652848,326.104483,50.559640,0.086107,327.103492,324.908658,34.701219,91.253023,0.354078,0.282573,-8103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,293.8000,293.8000,293.5000,94,293.6900,2025-03-01 00:55:00+00:00,2423,293.694392,293.599243,61.442065,0.101318,293.908383,293.514467,17.641309,8.570821,0.140146,0.054488,58505
3875,294.1365,294.1800,293.8001,287,293.8100,2025-03-01 00:56:00+00:00,12443,293.922855,293.623860,67.574960,0.125233,294.003721,293.467779,18.542093,226.530358,0.157279,0.165673,70948
3876,294.1187,294.1365,293.9152,154,294.1365,2025-03-01 00:57:00+00:00,5124,294.068604,293.648817,66.991875,0.141122,294.070852,293.449518,19.378535,190.686243,0.161852,0.152791,65824
3877,294.1507,294.1800,294.0500,149,294.1200,2025-03-01 00:58:00+00:00,5991,294.121301,293.674173,67.513198,0.154516,294.135043,293.433397,20.365429,189.103079,0.159576,0.136409,71815


In [29]:
data.to_csv('data.csv', index=False)

In [30]:
def normalize_dataframe_with_mean_std(df):
    """
    Normalize all columns in the dataframe except the 'time' column using Z-score normalization.
    
    Args:
        df (pd.DataFrame): Input dataframe.
        
    Returns:
        pd.DataFrame: Normalized dataframe.
        dict: Mean and standard deviation values for each column (used for normalizing new rows).
    """
    # Create a copy of the dataframe to avoid modifying the original
    normalized_df = df.copy()
    
    # Dictionary to store mean and standard deviation values for each column
    normalization_params = {}
    
    # Normalize all columns except 'time'
    for col in normalized_df.columns:
        if col != 'time':
            col_mean = normalized_df[col].mean()
            col_std = normalized_df[col].std()
            normalization_params[col] = {'mean': col_mean, 'std': col_std}
            
            # Apply Z-score normalization
            normalized_df[col] = (normalized_df[col] - col_mean) / col_std
        else :
            for i in range(len(normalized_df[col])) :
                normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute
            
            
    
    return normalized_df, normalization_params


def normalize_new_row_with_mean_std(row, normalization_params):
    """
    Normalize a new row using previously computed mean and standard deviation values.
    
    Args:
        row (pd.Series): New row of data.
        normalization_params (dict): Dictionary containing mean and standard deviation values for each column.
        
    Returns:
        pd.Series: Normalized row.
    """
    normalized_row = row.copy()
    
    for col in row.index:
        if col in normalization_params:
            col_mean = normalization_params[col]['mean']
            col_std = normalization_params[col]['std']
            
            # Apply Z-score normalization
            normalized_row[col] = (row[col] - col_mean) / col_std
        elif col == 'time':
            normalized_row[col] = row[col].hour*60 + row[col].minute
    
    return normalized_row

In [11]:
temp = df.iloc[0]
temp

close                          327.05
high                           327.05
low                             326.9
trades                             94
open                         326.9811
time        2025-02-25 00:00:00+00:00
volume                           2175
vwap                       326.998533
MA50                              NaN
RSI                               NaN
MACD                              NaN
BB_upper                          NaN
BB_lower                          NaN
ADX                               0.0
CCI                               NaN
ATR                               0.0
ROC                               NaN
OBV                              2175
Name: 0, dtype: object

In [27]:
normalized_data,normailzed_param =  normalize_dataframe_with_mean_std(data)
normalized_data

C:\Users\srija\AppData\Local\Temp\ipykernel_17080\2794413969.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute
C:\Users\srija\AppData\Local\Temp\ipykernel_17080\2794413969.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29' has dtype incompatible with datetime64[ns, UTC], please explicitly cast to a compatible dtype first.
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute


,close,high,low,trades,open,time,volume,vwap,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
29,2.158277,2.152984,2.170351,-0.553999,2.152767,29,-0.427367,2.162528,2.102913,0.469901,-0.089935,2.044892,2.112228,1.252026,1.751998,-0.588077,0.306275,1.897733
30,2.177495,2.166196,2.164500,-0.576687,2.147639,30,-0.420061,2.175541,2.102578,0.873834,0.060279,2.057040,2.106197,1.235946,1.753164,-0.564747,0.705883,1.898882
31,2.167805,2.163260,2.183514,-0.584249,2.179141,31,-0.439596,2.173530,2.101557,0.626269,0.159421,2.066402,2.103861,1.221016,1.492376,-0.597422,0.851050,1.898390
32,2.172936,2.166196,2.171082,-0.588374,2.153500,32,-0.442947,2.173684,2.100803,0.732397,0.246602,2.075723,2.101928,1.138645,1.248023,-0.589726,0.856293,1.898770
33,2.147283,2.158856,2.163038,-0.574281,2.174745,33,-0.439263,2.158009,2.098980,0.119493,0.262802,2.079673,2.102576,1.020032,0.886375,-0.580768,0.548740,1.898266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,-0.249399,-0.270612,-0.250243,-0.590093,-0.258214,55,-0.444866,-0.257164,-0.271604,1.445989,0.290322,-0.344639,-0.175672,-0.647692,0.150490,-1.123248,0.148767,1.906930
3875,-0.224736,-0.242721,-0.228297,-0.523749,-0.249423,56,-0.406091,-0.240423,-0.269808,2.193549,0.333588,-0.337676,-0.179060,-0.559634,2.090366,-1.079804,0.343743,1.908548
3876,-0.226041,-0.245914,-0.219879,-0.569468,-0.225504,57,-0.434414,-0.229742,-0.267988,2.122475,0.362335,-0.332773,-0.180386,-0.477866,1.771348,-1.068208,0.321151,1.907882
3877,-0.223695,-0.242721,-0.210021,-0.571187,-0.226713,58,-0.431059,-0.225881,-0.266139,2.186021,0.386567,-0.328085,-0.181555,-0.381390,1.757257,-1.073977,0.292423,1.908661


In [28]:
for i in range(len(normalized_data)) :
    print(normalized_data['time'].iloc[i])

29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
670
671
672
673
674
675
676
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
710
711
712
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770

In [40]:
(data['time'].iloc[0].date() - data['time'].iloc[1000].date()).days

-1

In [14]:
new_temp = normalize_new_row_with_mean_std(data.iloc[0],normailzed_param)
new_temp

close                        2.158277
high                         2.152984
low                          2.170351
trades                      -0.553999
open                         2.152767
time        2025-02-25 00:29:00+00:00
volume                      -0.427367
vwap                         2.162528
MA50                         2.102913
RSI                          0.469901
MACD                        -0.089935
BB_upper                     2.044892
BB_lower                     2.112228
ADX                          1.252026
CCI                          1.751998
ATR                         -0.588077
ROC                          0.306275
OBV                          1.897733
Name: 29, dtype: object

In [ ]:
def getTechnicalIndicators(data):
    # Calculate technical indicators
    data['MA50'] = ta.trend.sma_indicator(data['close'], window=30)
    data['RSI'] = ta.momentum.rsi(data['close'], window=30)
    data['MACD'] = ta.trend.macd(data['close'])
    data['BB_upper'] = ta.volatility.bollinger_hband(data['close'])
    data['BB_lower'] = ta.volatility.bollinger_lband(data['close'])
    data['ADX'] = ta.trend.adx(data['high'], data['low'], data['close'])
    data['CCI'] = ta.trend.cci(data['high'], data['low'], data['close'])
    data['ATR'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'])
    data['ROC'] = ta.momentum.roc(data['close'])
    data['OBV'] = ta.volume.on_balance_volume(data['close'], data['volume'])

    # Drop rows with NaN values (resulting from the indicator calculation)
    data = data.dropna()
    return data

In [5]:
def getNextStateBuySide(current_state,target,stop_loss,df):
    current_price = current_state['close']
    current_date = current_state['time'].date()
    target_price = current_price*target
    stop_loss_price = current_price*stop_loss
    
    for i in range(len(df)):
        if df['time'][i].date() == current_date:
            if df['high'][i] >= target_price:
                result = df.iloc[i].copy()
                result['action'] = 'Target Hit'
                result['delay'] = (-current_date + df['time'][i].date()).days*3600 + (-current_state['time'].hour + df['time'][i].hour)*60 + (-current_state['time'].minute + df['time'][i].minute)
                return result
            if df['low'][i] <= stop_loss_price:
                result = df.iloc[i].copy()
                result['action'] = 'Stop Loss Hit'
                result['delay'] = (-current_date + df['time'][i].date()).days*3600 + (-current_state['time'].hour + df['time'][i].hour)*60 + (-current_state['time'].minute + df['time'][i].minute)
                return result
        else:
            result = df.iloc[i-1].copy()
            result['action'] = 'End of Day'
            result['delay'] = (-current_date + df['time'][i].date()).days*3600 + (-current_state['time'].hour + df['time'][i].hour)*60 + (-current_state['time'].minute + df['time'][i].minute)  
            return result
    result = df.iloc[-1].copy()
    result['action'] = 'End of Day'
    return result
def getNextStateSellSide(current_state,target,stop_loss,df):
    current_price = current_state['close']
    current_date = current_state['time'].date()
    target_price = current_price*target
    stop_loss_price = current_price*stop_loss
    for i in range(len(df)):
        if df['time'][i].date() == current_date:
            if df['low'][i] <= target_price:
                result = df.iloc[i].copy()
                result['action'] = 'Target Hit'
                result['delay'] = (-current_date + df['time'][i].date()).days*3600 + (-current_state['time'].hour + df['time'][i].hour)*60 + (-current_state['time'].minute + df['time'][i].minute)
                return result
            if df['high'][i] >= stop_loss_price:
                result = df.iloc[i].copy()
                result['action'] = 'Stop Loss Hit'
                result['delay'] = (-current_date + df['time'][i].date()).days*3600 + (-current_state['time'].hour + df['time'][i].hour)*60 + (-current_state['time'].minute + df['time'][i].minute)
                return result
        else:
            result = df.iloc[i-1].copy()
            result['action'] = 'End of Day'
            result['delay'] = (-current_date + df['time'][i].date()).days*3600 + (-current_state['time'].hour + df['time'][i].hour)*60 + (-current_state['time'].minute + df['time'][i].minute)
            return result 
    result = df.iloc[-1].copy()
    result['action'] = 'End of Day'
    return result  
def generateTargetDataBuySide(df,target,stop_loss):
    target_data = []
    for i in range(len(df)):
        target_data.append(getNextStateBuySide(df.loc[i],target,stop_loss,df.iloc[i:].reset_index(drop=True)))
    df = pd.DataFrame(target_data)
    df.reset_index(drop=True,inplace=True)
    return df
def generateTargetDataSellSide(df,target,stop_loss):
    target_data = []
    for i in range(len(df)):
        target_data.append(getNextStateSellSide(df.loc[i],target,stop_loss,df.iloc[i:].reset_index(drop=True)))
    df = pd.DataFrame(target_data)
    df.reset_index(drop=True,inplace=True)
    return df

In [6]:
target_data = generateTargetDataBuySide(df,1.04,0.94)


In [7]:
target_data

,close,high,low,trades,open,time,volume,vwap,action
433,307.31,309.07,307.14,15356,308.9301,2025-02-25 15:17:00+00:00,857100,307.973231,Stop Loss Hit
432,307.31,309.07,307.14,15356,308.9301,2025-02-25 15:17:00+00:00,857100,307.973231,Stop Loss Hit
431,307.31,309.07,307.14,15356,308.9301,2025-02-25 15:17:00+00:00,857100,307.973231,Stop Loss Hit
430,307.31,309.07,307.14,15356,308.9301,2025-02-25 15:17:00+00:00,857100,307.973231,Stop Loss Hit
429,307.31,309.07,307.14,15356,308.9301,2025-02-25 15:17:00+00:00,857100,307.973231,Stop Loss Hit
...,...,...,...,...,...,...,...,...,...
4,294.40,294.45,294.16,316,294.1600,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3,294.40,294.45,294.16,316,294.1600,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
2,294.40,294.45,294.16,316,294.1600,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
1,294.40,294.45,294.16,316,294.1600,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day


In [128]:
target_data.reset_index(drop=True,inplace=True)
target_data

,close,high,low,trades,open,time,volume,vwap,action
0,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Stop Loss Hit
1,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Stop Loss Hit
2,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Stop Loss Hit
3,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Stop Loss Hit
4,312.730,314.209,312.66,16919,314.00,2025-02-25 14:52:00+00:00,947388,313.331252,Stop Loss Hit
...,...,...,...,...,...,...,...,...,...
3874,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3875,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3876,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3877,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day


In [131]:
len(target_data[target_data['action'] == 'End of Day'])

2624

In [148]:
target_data2 = generateTargetDataSellSide(df,0.96,1.06)


In [149]:
target_data2.reset_index(drop=True,inplace=True)
target_data2

,close,high,low,trades,open,time,volume,vwap,action
0,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Target Hit
1,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Target Hit
2,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Target Hit
3,314.011,315.900,313.90,20135,315.71,2025-02-25 14:51:00+00:00,909256,314.733036,Target Hit
4,312.730,314.209,312.66,16919,314.00,2025-02-25 14:52:00+00:00,947388,313.331252,Target Hit
...,...,...,...,...,...,...,...,...,...
3874,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3875,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3876,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day
3877,294.400,294.450,294.16,316,294.16,2025-03-01 00:59:00+00:00,17177,294.375250,End of Day


In [151]:
len(target_data2[target_data2['action'] == 'Target Hit'])

1247

: 

In [146]:
len(target_data[target_data['action']=='Stop Loss Hit'])

1247

In [135]:
df

,close,high,low,trades,open,time,volume,vwap
0,327.0500,327.0500,326.9000,94,326.9811,2025-02-25 00:00:00+00:00,2175,326.998533
1,327.2000,327.2200,327.0500,71,327.0600,2025-02-25 00:01:00+00:00,1644,327.132968
2,327.1600,327.2500,327.1000,83,327.1500,2025-02-25 00:02:00+00:00,3368,327.211843
3,327.2500,327.2500,327.0800,65,327.1700,2025-02-25 00:03:00+00:00,1367,327.161614
4,326.8218,327.2500,326.8000,176,327.2500,2025-02-25 00:04:00+00:00,6807,326.999470
...,...,...,...,...,...,...,...,...
3874,293.8000,293.8000,293.5000,94,293.6900,2025-03-01 00:55:00+00:00,2423,293.694392
3875,294.1365,294.1800,293.8001,287,293.8100,2025-03-01 00:56:00+00:00,12443,293.922855
3876,294.1187,294.1365,293.9152,154,294.1365,2025-03-01 00:57:00+00:00,5124,294.068604
3877,294.1507,294.1800,294.0500,149,294.1200,2025-03-01 00:58:00+00:00,5991,294.121301


In [91]:
df['close'][0]

327.05

In [96]:
df[0:]

,close,high,low,trades,open,time,volume,vwap
0,327.0500,327.0500,326.9000,94,326.9811,2025-02-25 00:00:00+00:00,2175,326.998533
1,327.2000,327.2200,327.0500,71,327.0600,2025-02-25 00:01:00+00:00,1644,327.132968
2,327.1600,327.2500,327.1000,83,327.1500,2025-02-25 00:02:00+00:00,3368,327.211843
3,327.2500,327.2500,327.0800,65,327.1700,2025-02-25 00:03:00+00:00,1367,327.161614
4,326.8218,327.2500,326.8000,176,327.2500,2025-02-25 00:04:00+00:00,6807,326.999470
...,...,...,...,...,...,...,...,...
3874,293.8000,293.8000,293.5000,94,293.6900,2025-03-01 00:55:00+00:00,2423,293.694392
3875,294.1365,294.1800,293.8001,287,293.8100,2025-03-01 00:56:00+00:00,12443,293.922855
3876,294.1187,294.1365,293.9152,154,294.1365,2025-03-01 00:57:00+00:00,5124,294.068604
3877,294.1507,294.1800,294.0500,149,294.1200,2025-03-01 00:58:00+00:00,5991,294.121301


In [117]:
df.iloc[0]['time'].date()

datetime.date(2025, 2, 25)

In [ ]:
import pandas as pd
import ta

# Load your stock data


# Calculate technical indicators
data['MA50'] = ta.trend.sma_indicator(data['close'], window=15)
data['RSI'] = ta.momentum.rsi(data['close'], window=15)
data['MACD'] = ta.trend.macd(data['close'])
data['BB_upper'] = ta.volatility.bollinger_hband(data['close'])
data['BB_lower'] = ta.volatility.bollinger_lband(data['close'])
data['ADX'] = ta.trend.adx(data['high'], data['low'], data['close'])
data['CCI'] = ta.trend.cci(data['high'], data['low'], data['close'])
data['ATR'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'])
data['ROC'] = ta.momentum.roc(data['close'])
data['OBV'] = ta.volume.on_balance_volume(data['close'], data['volume'])

# Drop rows with NaN values (resulting from the indicator calculation)
data = data.dropna()

# Add more indicators as needed

# Drop rows with NaN values (resulting from the indicator calculation)
historical_data = data

In [1]:
historical_data

NameError: name 'historical_data' is not defined

In [30]:
from utils_data import generateTargetDataBuySide,generateTargetDataSellSide

In [2]:
from utils_data import StockDataFetcher

In [3]:
fetcher = StockDataFetcher()

stock_symbol = "TSLA"
start_date = "2025-02-25T00:00:00Z"
end_date = "2025-03-02T09:30:00-04:00"

df = fetcher.get_stock_data(stock_symbol, start_date, end_date)

c:\Users\srija\Assignment\Trading\utils_data.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_market_hours.drop(columns=['time'], inplace=True)
c:\Users\srija\Assignment\Trading\utils_data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_market_hours.rename(columns={'time_et': 'time'}, inplace=True)


In [4]:
df

,close,high,low,trades,open,volume,vwap,time
386,325.4500,328.8900,325.1000,18530,327.025,1175100,326.455812,2025-02-25 09:30:00-05:00
387,321.9300,325.7500,321.4300,25068,325.430,1130193,323.352600,2025-02-25 09:31:00-05:00
388,323.9929,324.3500,321.8800,11287,321.940,688359,323.324104,2025-02-25 09:32:00-05:00
389,323.3000,324.3100,322.5900,9058,324.005,402245,323.392859,2025-02-25 09:33:00-05:00
390,324.7901,325.2699,323.1900,9012,323.220,535038,324.513255,2025-02-25 09:34:00-05:00
...,...,...,...,...,...,...,...,...
3637,291.7250,292.1800,291.5115,5086,291.650,337417,291.848434,2025-02-28 15:56:00-05:00
3638,292.3290,292.3297,291.4700,6071,291.680,390982,292.021535,2025-02-28 15:57:00-05:00
3639,293.1100,293.1600,292.1400,8539,292.320,654559,292.754669,2025-02-28 15:58:00-05:00
3640,293.3100,293.6200,292.9700,14705,293.120,1348984,293.269243,2025-02-28 15:59:00-05:00


In [5]:
from utils_data import getTechnicalIndicators, generateTargetDataBuySide

In [6]:
df = getTechnicalIndicators(df)

In [7]:
df

,close,high,low,trades,open,volume,vwap,time,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
415,313.100,313.7800,312.9101,8115,313.0500,456001,313.271021,2025-02-25 09:59:00-05:00,318.244920,29.349700,-2.245836,321.045435,311.418325,63.167661,-98.274278,1.746396,-1.550168,-5399034
416,311.515,313.2000,311.0000,20581,313.2000,957038,311.709511,2025-02-25 10:00:00-05:00,317.780420,26.877466,-2.345220,320.777912,310.867848,62.931757,-128.432951,1.778796,-1.662037,-6356072
417,311.000,311.7400,310.4500,12553,311.4600,712770,310.741341,2025-02-25 10:01:00-05:00,317.416087,26.137435,-2.437441,320.295853,310.422057,62.855160,-142.196975,1.743882,-1.585393,-7068842
418,311.420,312.6700,310.7410,10361,310.9043,552196,311.829739,2025-02-25 10:02:00-05:00,316.996990,27.814225,-2.448413,319.604013,310.232897,61.824678,-116.467426,1.757105,-1.414675,-6516646
419,310.950,311.5699,310.1000,12722,311.4900,572668,310.708111,2025-02-25 10:03:00-05:00,316.585323,27.101984,-2.466600,319.283227,309.824673,61.081526,-129.257740,1.736590,-0.974807,-7089314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,291.725,292.1800,291.5115,5086,291.6500,337417,291.848434,2025-02-28 15:56:00-05:00,290.986730,55.584289,0.063786,292.105772,289.021738,20.630417,153.435981,1.052665,0.385334,-5893025
3638,292.329,292.3297,291.4700,6071,291.6800,390982,292.021535,2025-02-28 15:57:00-05:00,291.006697,57.637442,0.183024,292.380913,288.967497,20.977128,151.293644,1.038882,0.667723,-5502043
3639,293.110,293.1600,292.1400,8539,292.3200,654559,292.754669,2025-02-28 15:58:00-05:00,291.037697,60.104320,0.336660,292.810814,288.797596,22.022453,186.929632,1.037533,0.933196,-4847484
3640,293.310,293.6200,292.9700,14705,293.1200,1348984,293.269243,2025-02-28 15:59:00-05:00,291.039290,60.710421,0.469149,293.222316,288.682094,23.338075,189.619328,1.009852,1.066451,-3498500


In [8]:
df_buy_side = generateTargetDataBuySide(df,1.04,0.94)
df_buy_side

Processing Buy Side Data: 100%|██████████| 10/10 [00:00<00:00, 18.50it/s]


,close,high,low,trades,open,volume,vwap,time,MA50,RSI,...,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV,action,delay,next state index
0,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1411.0,362
1,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1410.0,362
2,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1409.0,362
3,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1408.0,362
4,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1407.0,362
5,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1406.0,362
6,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1405.0,362
7,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1404.0,362
8,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1403.0,362
9,302.99,303.04,302.55,513,302.745,5943602,302.799933,2025-02-25 16:00:00-05:00,301.136343,56.03219,...,304.029135,298.006065,27.14731,99.199705,0.870763,1.312424,-8079635,End of Day,1402.0,362


In [9]:
from utils_data import normalize_dataframe_with_mean_std
df_norm = normalize_dataframe_with_mean_std(df)
df_norm

c:\Users\srija\Assignment\Trading\utils_data.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540
c:\Users\srija\Assignment\Trading\utils_data.py:201: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '59' has dtype incompatible with datetime64[ns, US/Eastern], please explicitly cast to a compatible dtype first.
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540


(        close      high       low    trades      open     volume      vwap  \
 415   313.100  313.7800  312.9101  1.299617  313.0500   0.550054  2.582928   
 416   311.515  313.2000  311.0000  5.138084  313.2000   1.994599  2.378359   
 417   311.000  311.7400  310.4500  2.666143  311.4600   1.290348  2.251521   
 418   311.420  312.6700  310.7410  1.991194  310.9043   0.827395  2.394109   
 419   310.950  311.5699  310.1000  2.718181  311.4900   0.886418  2.247168   
 ...       ...       ...       ...       ...       ...        ...       ...   
 3637  291.725  292.1800  291.5115  0.366943  291.6500   0.208163 -0.223588   
 3638  292.329  292.3297  291.4700  0.670239  291.6800   0.362597 -0.200911   
 3639  293.110  293.1600  292.1400  1.430173  292.3200   1.122519 -0.104865   
 3640  293.310  293.6200  292.9700  3.328776  293.1200   3.124623 -0.037452   
 3641  293.820  293.8800  292.9700 -0.932458  293.8800  24.636385 -0.075154   
 
      time      MA50       RSI      MACD  BB_upper

In [10]:
df_norm[0]

,close,high,low,trades,open,volume,vwap,time,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
415,313.100,313.7800,312.9101,1.299617,313.0500,0.550054,2.582928,59,3.133509,-2.320201,-2.938210,3.222563,2.539589,4.090891,-0.754937,2.353980,-1.890103,1.736895
416,311.515,313.2000,311.0000,5.138084,313.2000,1.994599,2.378359,60,3.073979,-2.629115,-3.076157,3.188411,2.469035,4.066180,-1.025792,2.448869,-2.031765,1.481874
417,311.000,311.7400,310.4500,2.666143,311.4600,1.290348,2.251521,61,3.027286,-2.721584,-3.204162,3.126872,2.411899,4.058157,-1.149407,2.346618,-1.934709,1.291943
418,311.420,312.6700,310.7410,1.991194,310.9043,0.827395,2.394109,62,2.973574,-2.512064,-3.219391,3.038553,2.387655,3.950214,-0.918330,2.385342,-1.718525,1.439086
419,310.950,311.5699,310.1000,2.718181,311.4900,0.886418,2.247168,63,2.920814,-2.601061,-3.244635,2.997602,2.335333,3.872369,-1.033200,2.325262,-1.161510,1.286488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,291.725,292.1800,291.5115,0.366943,291.6500,0.208163,-0.223588,416,-0.359918,0.957901,0.267593,-0.471832,-0.330949,-0.364877,1.505671,0.322287,0.560865,1.605262
3638,292.329,292.3297,291.4700,0.670239,291.6800,0.362597,-0.200911,417,-0.357359,1.214449,0.433098,-0.436708,-0.337901,-0.328559,1.486430,0.281921,0.918461,1.709446
3639,293.110,293.1600,292.1400,1.430173,292.3200,1.122519,-0.104865,418,-0.353386,1.522694,0.646349,-0.381827,-0.359677,-0.219061,1.806477,0.277971,1.254635,1.883866
3640,293.310,293.6200,292.9700,3.328776,293.1200,3.124623,-0.037452,419,-0.353182,1.598429,0.830245,-0.329296,-0.374480,-0.081250,1.830633,0.196903,1.423379,2.243328


In [10]:
df[0]['time'].min()

30

In [37]:
import pytz

# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Convert to Eastern Time (ET)
df['time_et'] = df['time'].dt.tz_convert('US/Eastern')

# Filter data between 09:30 AM and 04:00 PM ET
df_market_hours = df[(df['time_et'].dt.time >= pd.to_datetime("09:30:00").time()) & 
                     (df['time_et'].dt.time <= pd.to_datetime("16:00:00").time())]

# Display filtered dataframe
print(df_market_hours)

C:\Users\srija\AppData\Local\Temp\ipykernel_30144\2364311740.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


         close      high       low  trades     open                      time  \
386   325.4500  328.8900  325.1000   18530  327.025 2025-02-25 14:30:00+00:00   
387   321.9300  325.7500  321.4300   25068  325.430 2025-02-25 14:31:00+00:00   
388   323.9929  324.3500  321.8800   11287  321.940 2025-02-25 14:32:00+00:00   
389   323.3000  324.3100  322.5900    9058  324.005 2025-02-25 14:33:00+00:00   
390   324.7901  325.2699  323.1900    9012  323.220 2025-02-25 14:34:00+00:00   
...        ...       ...       ...     ...      ...                       ...   
3637  291.7250  292.1800  291.5115    5086  291.650 2025-02-28 20:56:00+00:00   
3638  292.3290  292.3297  291.4700    6071  291.680 2025-02-28 20:57:00+00:00   
3639  293.1100  293.1600  292.1400    8539  292.320 2025-02-28 20:58:00+00:00   
3640  293.3100  293.6200  292.9700   14705  293.120 2025-02-28 20:59:00+00:00   
3641  293.8200  293.8800  292.9700     866  293.880 2025-02-28 21:00:00+00:00   

       volume        vwap  

C:\Users\srija\AppData\Local\Temp\ipykernel_30144\2364311740.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_et'] = df['time'].dt.tz_convert('US/Eastern')


In [38]:
df_market_hours

,close,high,low,trades,open,time,volume,vwap,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV,time_et
386,325.4500,328.8900,325.1000,18530,327.025,2025-02-25 14:30:00+00:00,1175100,326.455812,328.650380,27.320329,-0.676726,330.548023,326.296157,24.992606,-178.177449,0.680608,-1.036915,-1556692,2025-02-25 09:30:00-05:00
387,321.9300,325.7500,321.4300,25068,325.430,2025-02-25 14:31:00+00:00,1130193,323.352600,328.419713,18.682740,-1.076579,331.571329,324.550851,26.542748,-315.221602,0.940564,-2.000000,-2686885,2025-02-25 09:31:00-05:00
388,323.9929,324.3500,321.8800,11287,321.940,2025-02-25 14:32:00+00:00,688359,323.324104,328.244633,31.762214,-1.213024,331.675575,323.916905,27.982165,-219.530443,1.049810,-1.462013,-1998526,2025-02-25 09:32:00-05:00
389,323.3000,324.3100,322.5900,9058,324.005,2025-02-25 14:33:00+00:00,402245,323.392859,328.045343,30.081028,-1.361376,331.735128,323.231362,29.318766,-175.205637,1.097681,-1.666768,-2400771,2025-02-25 09:33:00-05:00
390,324.7901,325.2699,323.1900,9012,323.220,2025-02-25 14:34:00+00:00,535038,324.513255,327.893680,37.446867,-1.343223,331.548195,322.947305,29.213193,-113.881990,1.167839,-1.213547,-1865733,2025-02-25 09:34:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,291.7250,292.1800,291.5115,5086,291.650,2025-02-28 20:56:00+00:00,337417,291.848434,290.986730,55.584317,0.063786,292.105772,289.021738,20.630417,153.435981,1.052665,0.385334,-12406317,2025-02-28 15:56:00-05:00
3638,292.3290,292.3297,291.4700,6071,291.680,2025-02-28 20:57:00+00:00,390982,292.021535,291.006697,57.637468,0.183024,292.380913,288.967497,20.977128,151.293644,1.038882,0.667723,-12015335,2025-02-28 15:57:00-05:00
3639,293.1100,293.1600,292.1400,8539,292.320,2025-02-28 20:58:00+00:00,654559,292.754669,291.037697,60.104346,0.336660,292.810814,288.797596,22.022453,186.929632,1.037533,0.933196,-11360776,2025-02-28 15:58:00-05:00
3640,293.3100,293.6200,292.9700,14705,293.120,2025-02-28 20:59:00+00:00,1348984,293.269243,291.039290,60.710447,0.469149,293.222316,288.682094,23.338075,189.619328,1.009852,1.066451,-10011792,2025-02-28 15:59:00-05:00


In [ ]:
df_buy_side = generateTargetDataBuySide(df_market_hours,1.04,0.94)

Processing Buy Side Data:   0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'int' object has no attribute 'date'

In [12]:
df_buy_side

NameError: name 'df_buy_side' is not defined

In [13]:
from utils_data import normalize_dataframe_with_mean_std
df_final = normalize_dataframe_with_mean_std(df_market_hours)
df_final

NameError: name 'df_market_hours' is not defined

In [12]:
df_final.iloc[0]

AttributeError: 'tuple' object has no attribute 'iloc'

In [11]:
df_buy_side.iloc[0]

close                                   306.47
high                                    308.77
low                                   306.2313
trades                                   13214
open                                   308.195
time                 2025-02-25 15:20:00+00:00
volume                                  721937
vwap                                307.275594
MA50                                  311.6687
RSI                                  28.064591
MACD                                 -2.055109
BB_upper                            314.241855
BB_lower                            307.060225
ADX                                  44.888423
CCI                                -174.572799
ATR                                   1.559026
ROC                                  -1.497766
OBV                                  -11625756
action                           Stop Loss Hit
delay                                    891.0
next state index                           407
Name: 0, dtyp

In [10]:
df.iloc[407]

close                          306.47
high                           308.77
low                          306.2313
trades                          13214
open                          308.195
time        2025-02-25 15:20:00+00:00
volume                         721937
vwap                       307.275594
MA50                         311.6687
RSI                         28.064591
MACD                        -2.055109
BB_upper                   314.241855
BB_lower                   307.060225
ADX                         44.888423
CCI                       -174.572799
ATR                          1.559026
ROC                         -1.497766
OBV                         -11625756
Name: 436, dtype: object

In [12]:
alpaca_api = "FMAHHZRVCBFJRPTM"